In [ ]:
# necessary imports
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests
import os

In [ ]:
# create driver instance
driver = webdriver.Firefox()

In [ ]:
def save_image(url,filename):
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)

In [ ]:
def scrape_images(term):
    
    # create a folder for the image in the current working directory
    try:
        path = os.mkdir(f'./{term}')
    except:
        pass
    
    # search for term and get the number of result pages
    driver.get(f'https://pixabay.com/photos/search/{term} animal/')
    pages = int(driver.find_element(By.CSS_SELECTOR,'div.hideSm--2N9ks:nth-child(3) > div:nth-child(1) > span:nth-child(2)').text.split()[-1])

    # initialize a variabe to keep track of total scraped images
    scraped = 0
    
    # save links of scraped images
    scraped_links = []
    
    for page in range(1,pages+1):
        
        # access results page and scroll to bottom to load all images on current page
        driver.get(f'https://pixabay.com/photos/search/{term} animal/?pagi={page}')
        time.sleep(5)
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(5)
        
        # access the results cntainer and get the link for all thumbnails
        results = driver.find_element(By.CSS_SELECTOR,'.container--HcTw2')
        image_links = [image.get_attribute('src') for image in results.find_elements(By.TAG_NAME,'img')]
        
        # navigate through the thumbnail links and save images into the desired folder
        for i in range(len(image_links)):
            url = image_links[i]
            save_image(url,f'{term}/{term}_{scraped}.jpg')
            scraped_links.append(url)
            scraped+=1
            
            # limit scraped images to 1200
            if scraped == 1200:
                break
        else:
            continue

        break
    print(f'scraped {scraped} {term} images from {page} page(s)')
    
    # create a .txt file to save thumbnail links of scraped images
    with open(f'{term}/{term} links.txt', 'a') as f:
        f.write('\n'.join(scraped_links))

In [ ]:
# scrape images for required animals
for term in ['buffalo','elephant','rhino','zebra']:
    scrape_images(term)